# Install Dependencies

In [ ]:
%display latex

In [ ]:
%pip install pycryptodome pwntools

# Challenge

In [ ]:
import hashlib, json
from Crypto.Util.number import getPrime, isPrime
from random import randint

In [ ]:
assert isPrime(FROST_PRIME := 0x1a66804d885939d7acf3a4b413c9a24547b876e706913adec9684cc4a63ab0dfd2e0fd79f683de06ad17774815dfc8375370eb3d0fb5dce0019bd0632e7663a41)
LIMIT = 2_000

In [ ]:
class TinselRNG:
    def __init__(self, bits):
        self.p = getPrime(bits)
        self.g = randint(1, self.p)

    def sparkle_bit(self):
        if self.g == 0:
            self.g += 1
        while True:
            shimmer = pow(self.g, (self.p-1)//2, self.p)
            yield int(shimmer == 1)
            self.g = (self.g + 1) % self.p
            if self.g == 0:
                self.g += 1

    def gather_sparkles(self, l):
        bits = ''
        for i, b in enumerate(self.sparkle_bit()):
            if i == l: break
            bits += str(b)
        return int(bits, 2)

In [ ]:
def frostscribe_signature(msg):
    blizzard = hashlib.sha512(msg.encode()).digest()
    snowmark = int.from_bytes(blizzard, "big") % FROST_PRIME
    lantern_key = frostrng.gather_sparkles(500)
    etch = pow(snowmark, lantern_key, FROST_PRIME)
    return {"signature": str(etch)}

# Solution

In [ ]:
from tqdm.auto import tqdm

In [ ]:
QUERY_SIZE = 500
KEY_SUBMISSION_SIZE = 84*8

WITH_HEAVY_ASSERTS = True
TEST_SERVER_PATH = "debug-server.py"

FALSE_FLAG = "HTB{fake_flag_for_testing}"

## Weakness 1: Order of `FROST_ORDER` is smooth as butter

Sagemath will crunch these discrete logarithms of $g^x \equiv b \bmod (F \coloneqq \text{FROST PRIME})$ quickly via pohlig-hellman.

### 'Smooth' group order: decomposable into small prime factors
- Exists in contrast to safe prime/sophie-germaine prime pairs of $(p,q)$, where $p - 1 = 2q$ (think of these as having a extraordinarily 'jagged' order ⛰️ - ergo, why these are so useful in cryptography, and why using your own primes w/o checking can shoot yourself in the foot!)

### Pohlig-hellman

TL;DR is for composite group orders, to augment your generic strategy with an initial a CRT-style decomposition to gain divide-and-conquer benefits

The step-by-step under-the-hood is:
- CRT-decompose `FROST_ORDER` into its prime-power subgroups
    - easily done with $g^x \equiv b \bmod F \implies (h_i \coloneqq g^\frac{F - 1}{{p_i}^{e_i}})^x$ $ \equiv b^\frac{F - 1}{{p_i}^{e_i}} \bmod F$
    - Note that these sub-cases of $(h_i)^x$, only have sub-order ${p_i}^{e_i}$ (not $F - 1$)
    - Thus, we are now dealing with equations not of $x \mod F - 1$, but its sub-cycles of $x_i \bmod {p_i}^{e_i}$ 
- Solve these smaller sub-cycle cases with a generic cyclic search strategy, such as:
    - Exhaustive search (self-explanatory)
    - Baby-step Giant-step (meet-in-the-middle table go brrr)
    - Pollard's rho (still the 🐐 of generic search strategies)
    - Pollard's lambda/🦘 (variant of rho, where you can control the search bounds)
- Then, recombine their results via CRT to get $x \bmod F - 1$!
    - The set of equations of $x_i \bmod {p_i}^{e_i}$ uniquely identify $x \bmod F - 1$
    - Recombination here is literally just the textbook CRT
    - Ergo, calculate your CRT basis for each ${p_i}^{e_i}$, scale by $x_i$, and you will get $x \bmod \prod_i {p_i}^{e_i}$, i.e. $x \bmod F - 1$!

In [ ]:
display(FROST_ORDER := factor(FROST_PRIME - 1), ceil(log(FROST_PRIME - 1, 2)))
assert max(p for p, e in FROST_ORDER) < 1 << 16
ZP_FROST = Zmod(FROST_PRIME)

## Weakness 2: Linear Legendre PRG has small state + seeds via secret offset + progresses sequentially

Linear Legrendre PRF/PRG is based on:
- Public parameters: $p$ (our prime field)
    - Public sequence: $L_p : (\mathbb{Z}/p\mathbb{Z})^* \to \{-1,1\} : i \mapsto \left(\dfrac{i}{p}\right)_\text{Leg}$
    - We then remap our output per $\{-1 \mapsto 0, 1 \mapsto 1\}$
- Private parameters $k$ (our secret offset)
    - Seed: $k \mapsto i \mapsto L_p(i + k)$

For this, we are just going to use a simple meet-in-the-middle strategy. This exploits for this PRNG, the sequence is public; just the offset is hidden. Thus, if your guess falls within the RNG stream, you can recover both the original seed and current state trivially by rewinding/progressing your index with a corresponding offset.

Thus, you can 'meet-in-the-middle' by finding overlapping candidates between the oracle RNG-stream substrings, and guessed-offset RNG-stream substrings.
- Then, validate each candidate by checking if their inferred seed-offset would generate the full oracle RNG-stream

Thus, this solution consists of:

1. Extract continuous PRG bit stream material from repeated signature queries
    - Since we know the message, we know $g$, and since $F$ is weak, we can discrete-log out the lantern key used
    - Ideally, we would get ~$\sqrt{p}$ bits of RNG material; but practically, we just get as much as possible

2. Create a lookup table to validate candidates against:
    - Keys: all $n$-length continuous substring tuples in the PRNG bit stream
    - Values: the substring offset (i.e. whatever index-arithmetic this substring has to the seed offset $k$)
        - NOTE: all offsets must have a unique substring; thus, minimum length is ~$\lceil \log_2(p) \rceil$
        - As such, redo with ($n \coloneqq n+1$)-length substrings if collision found, until minimum collision-free substring dictionary is found

3. Sample candidate $L_p$ substrings with random-offsets $r$ and length $n$, and check if $\|_{i=0}^{n-1} L_p(r + i)$ matches a substring in the lookup table

    - $O(1)$ hash-map makes initial pruning of offset guesses fast and independent from how much RNG material we collected
    - If a match is found, then derive the inferred $k^\prime$ offset, and check if $k^\prime$ yields the same full RNG-stream as observed
        - If no - go back to guessing
        - If yes - boom, we have recoverd $k$!

4. Now, we can solve any RNG challenges we need to get the flag!

### Weakness 2.5: Legrendre is multiplicative - thus, can derive $M^2$ solution offsets from a $M$-length sequential RNG stream

Interest in the Legrendre PRF recently spiked when the etherium foundation started exploring using the Legrendre in their protocols. Since then, a ton of optimized breaking strategies on this have been researched and published:

- <https://eprint.iacr.org/2020/098.pdf>
    - Note pages 5-8 for a direct outline of improvements applicable for this challenge
- <https://eprint.iacr.org/2019/1357.pdf>
 
These improvements mainly consist of ways to extract more substring + index-relations from a sequential Legrendre sequence:
- This is done by using the Legrendre symbol's multiplicative properties to infer, given a length $M$ RNG-stream value, substrings at:
    - $k + i$ offset with length $M - i$
        - This is just the seeded-by-offset weakness, expressed formally as $\left(\dfrac{(k + \epsilon) + i}{p}\right)_\text{Leg} = \left(\dfrac{k + (i + \epsilon)}{p}\right)_\text{Leg}$
        - i.e. if we guess $k + \epsilon \land \epsilon \leq M$, we match to a substring, and derive the candidate per $k = (k + \epsilon) - \epsilon$
    - $-k$ offset with length $M$
        - per $\left(\dfrac{k + i}{p}\right)_\text{Leg} \left(\dfrac{-1}{p}\right)_\text{Leg} = \left(\dfrac{-k - i}{p}\right)_\text{Leg}$
        - i.e. Legrendre reflects across $\mathbb{Z} / p\mathbb{Z}$ by multiplication with $\left(\dfrac{-1}{p}\right)_\text{Leg}$; so, can likewise reflect Legrendre sequences to get their negation
    - $\frac{k}{d}$ offsets with length $\lfloor \frac{M - 1}{d} \rfloor$
        - per $\left(\dfrac{\frac{k}{d} + i}{p}\right)_\text{Leg} \left(\dfrac{d}{p}\right)_\text{Leg} = \left(\dfrac{k + di}{p}\right)_\text{Leg}$
        - i.e. linear subsequences of $(k + \epsilon) + di$ within $k + i$ can be rescaled into sequential sequences of $\frac{k + \epsilon}{d} + i$

Thus, we can beef up our substring dictionary with more matches and candidate relationships to $k$
- Not only is now the success chance of each guess increased
- But the rate at which guesses improve grows quadratically, not linearly

Then, you can also likewise augment the search by not guessing singular sub-strings, but rather guessing larger RNG stream chunks of size $N$.
- Then, you can yield $O(N^2)$ substrings guesses w/o any new Legrendre-symbol calls
- However, substrings of $N$ are interdependent - pages 7-8 cover decomposition strategies of $N$ into guessesthat try to avoid computing overlapping comparisons
    - e.g. don't both negating across $\mathbb{Z} / p\mathbb{Z}$, skip implicitly-eliminated shifts, scale by primes to avoid overlapping rescaling factors

However, all of this is overkill for the challenge, so will leave this as a fun TODO for later

## RNG-stream generators

In [ ]:
def gen_rand_offsets(p):
    Zpp = GF(p)
    while True:
        b = Zpp.random_element()
        if not b.is_zero():        
            yield b

In [ ]:
def gen_rng_stream(p, offset, size, backwards=False):
    p = int(p)
    offset = int(offset)
    size = int(size)
    
    yield from ((
            max(legendre_symbol(idx, p), 0)
            if ((idx := 1 + ((offset + (-i if backwards else i) - 1) % (p - 1))) != 0) 
            else ValueError(
                "Error in rng_stream - this is strange, since my offset arithmetic *should* mean zero never occurs"
                f"\n{i=}\n{idx=}\n\n{p=}\n{offset=}\n{size=}"
        )) for i in range(size)
    )

def rng_stream_to_int(it):
    state = int(0)
    for b in it:
        state = state << int(1)
        state += int(b)
    return state

def int_to_rng_stream(i, size):
    return ZZ(i).digits(2, padto=size)[::-1]

def gen_rng_int(p, offset, size):
    return rng_stream_to_int(gen_rng_stream(p, offset, size))

### Asserts to check rng_stream generators

In [ ]:
# There are a bunch of equivalent methods for computing the internal legendre state
# this shows all the ones I explored, and that they match the Tinsel RNG bit-by-bit
_assert_rng = TinselRNG(16) # 64

_tries = 1 << (
    18 # Using tries > 4 * |p|, so slices wrapping around $p$ to $1$ are tested
    if WITH_HEAVY_ASSERTS
    else 8
)
_size = 16

In [ ]:
_start_g = _assert_rng.g
assert all((
    _assert_rng.gather_sparkles(1)
    == max(legendre_symbol(_offset, _assert_rng.p), 0)
    == max(kronecker(_offset, _assert_rng.p), 0)
    == int(Zmod(_assert_rng.p)(_offset).is_square())
    ) for i in tqdm(range(_tries), desc="Validating legendre rng_stream methods")
    if (_offset := (_start_g + i) % _assert_rng.p) != 0
)
del _start_g, _offset

In [ ]:
_start_g = _assert_rng.g
assert all((
    _assert_rng.gather_sparkles(_size)
    == rng_stream_to_int(_k)
    == int(''.join(str(b) for b in _k), 2)
    == ZZ(_k[::-1], 2)
    ) for i in tqdm(range(_tries), desc="Validating rng_stream -> int methods")
    if (_k := tuple(gen_rng_stream(_assert_rng.p, _start_g + (i * _size), _size))) is not None
), f"{_assert_rng}\n{_assert_rng.p}\n{_assert_rng.g}\n\n{i=}\n{type(i)=}\n{_k=}"
del _start_g, _k

In [ ]:
_start_g = _assert_rng.g
assert all(
    _assert_rng.gather_sparkles(_size) == (_k := gen_rng_int(_assert_rng.p, _start_g + (i * _size), _size))
    for i in tqdm(range(_tries), desc="Validating rng_int method")
), f"{_assert_rng}\n{_assert_rng.p}\n{_assert_rng.g}\n\n{i=}\n{type(i)=}\n{_k=}"
del _start_g, _k

In [ ]:
assert all((
    r == rng_stream_to_int(int_to_rng_stream(r, _size))
    ) for r in tqdm(range(1 << _size), desc="Validating int -> rng_stream methods")
)

In [ ]:
del _assert_rng, _tries, _size

## Substring Dictionary Builders

In [ ]:
def tqdm_remover(bar):
    # <https://stackoverflow.com/a/79707132>
    # bar.container.close()
    bar.leave = False
    bar.n = bar.total
    bar.close()

In [ ]:
from collections import deque
from itertools import islice

def sliding_window(iterable, n):
    it = iter(iterable)
    d = deque(islice(it, n), maxlen=int(n))
    if len(d) == n:
        yield tuple(d)
    for x in it:
        d.append(x)
        yield tuple(d)

In [ ]:
def get_unique_substring_dict(it, *, max_len=128, clear_bar=True):
    seq = tuple(it)
    num_states = len(set(seq))

    min_len = int(ceil(log(len(seq), num_states))) # ~approx pigenhole principle for seq to be feasible

    for n in tqdm(
        range(min_len, max_len + 1),
        desc="Trialling substring lengths for unique index dictionary",
        initial=int(min_len),
        total=int(max_len),
        # position=0
    ):
        subseq_dict = {}
        for idx, subseq in (inner_tqdm := tqdm(
            enumerate(sliding_window(seq, n)),
            desc=f"Trial - {n=}",
            initial=int(n-1),
            total=int(len(seq)),
            # position=1,
            # leave=False,
        )):
            if subseq in subseq_dict:
                if clear_bar: tqdm_remover(inner_tqdm)
                break
            subseq_dict[subseq] = idx
        else:
            assert len(seq) == (n + len(subseq_dict) - 1)
            assert all(v == seq[i:i+n] for v, i in subseq_dict.items())
            return n, subseq_dict
    return None, None

def subint_dict(subseq_dict):
    return {rng_stream_to_int(k): v for k, v in tqdm(subseq_dict.items(), desc="parsing_to_int")}

# def get_unique_subint_dict(it, **kargs):
#     n, subseq_dict = get_unique_substring_dict(it, **kargs)
#     return n, subint_dict(subseq_dict)

if WITH_HEAVY_ASSERTS:
    from random import randint
    seq_n, seq_dict = get_unique_substring_dict((randint(0, 1) for _ in range(1 << 20)), max_len=64)
    print("done!")

In [ ]:
from itertools import islice

def gen_candidates(p, rng_stream_dict, max_samples, sample_size, use_int = False):
    Zpp = Zmod(p)
    max_samples = int(max_samples)

    if use_int:
        rng_int_dict = subint_dict(rng_stream_dict)
    
    yield from (
        Zpp(cand_idx - offset)
        for cand_idx in tqdm(islice(gen_rand_offsets(p), max_samples), total=max_samples)
        if (offset := (
            rng_int_dict.get(gen_rng_int(p, cand_idx, sample_size))
            if use_int else
            rng_stream_dict.get(tuple(gen_rng_stream(p, cand_idx, sample_size)))
        )) is not None
    )

## Server interactions

In [ ]:
def extract_chal_rng_stream(
    con,
    msg: str,
):
    con.recvuntil(b"> ")
    con.sendline(b"1")

    msg_g = int.from_bytes(hashlib.sha512(msg.encode()).digest(), "big")
    con.recvuntil(b"message: ")
    con.sendline(msg.encode())
    sig = json.loads(con.recvline().decode())['signature']

    sig_key = discrete_log(ZP_FROST(sig), ZP_FROST(msg_g))

    key_bits = ZZ(sig_key).digits(2, padto=QUERY_SIZE)[::-1]
    return key_bits

In [ ]:
def submit_guess(
    con,
    guess: str,
):
    con.recvuntil(b"> ")
    con.sendline(b"2")
    con.recvuntil(b"Reveal my snow-otp (in bits): ")
    con.sendline(guess.encode())
    if (flag := json.loads(con.recvline())['starshard']) == FALSE_FLAG:
        return None
    return flag

## Challenge Solver

In [ ]:
import pwn
from itertools import islice

from typing import Callable, TypeAlias

RngSolver: TypeAlias = Callable[[list[int], int], int | None]

def solve_rng(
    rng_stream,
    p,
    /,
    max_sample_size = 1 << 10,
    max_samples = 1 << 20,
    use_int_dict = False,
) -> int | None:
    rng_stream_len = len(rng_stream)

    with pwn.log.progress("Building rng_stream window dictionary") as plog:
        sample_size, rng_stream_dict = get_unique_substring_dict(rng_stream, max_len=max_sample_size)

        if sample_size is None:
            plog.failure(f"key stream dict could not be built - loosen the {max_sample_size=}")
            return None
        plog.status(f"{sample_size=}")

    with pwn.log.progress("Offset candidate trials") as plog:
        for cand_seed in gen_candidates(p, rng_stream_dict, max_samples, sample_size, use_int=use_int_dict):
            plog.status(f"Candidate found! {cand_seed=}")
            if any(
                exp != obs
                for exp, obs
                in zip(rng_stream, gen_rng_stream(p, cand_seed, rng_stream_len))
            ):
                plog.status("Candidate was a mismatch - continuing...")
                continue
            
            plog.success(f"Candidate is a match! {cand_seed=}")
            return cand_seed
        else:
            plog.failure("We could not find a solution! increase max_samples, or collect more bits")
            return None
    

In [ ]:
def solve_chal(
    queries = LIMIT - 1, # 800, # 50, # 
    query_msg = 'const_msg',

    solver_func: RngSolver = solve_rng,
    # max_sample_size = 128, # 48, # 24, # 32, # 100, # 
    # max_samples = 5_000_000, # 400_000, # 5000, # 

    run_local_debug=False,
    ip=None, 
    port=None,
):
    if not run_local_debug:
        assert ip is not None
        assert port is not None

    with (
        pwn.remote(ip, port)
        if not run_local_debug
        else pwn.process(["sage", "-python", TEST_SERVER_PATH])
    ) as con:
        
        with pwn.log.progress("Setting up challenge") as plog:
            con.recvuntil(b"frostrng.holly_prime = ")
            p = ZZ(con.recvline().decode().strip())
            Zpp = Zmod(p)
            plog.status(f"{p=}")
            if run_local_debug:
                g = con.recvline().decode().strip("frostrng.sleigh_seed = ").rstrip()
                plog.status(f"{g=}")

        with pwn.log.progress("Gathering rng_stream bits") as plog:
            rng_stream = [
                bit
                for _ in tqdm(range(queries))
                for bit in extract_chal_rng_stream(con, query_msg)
            ]
            assert (rng_stream_len := (queries * QUERY_SIZE)) == len(rng_stream)
            plog.status(f"rng_size: {rng_stream_len:_}")

        with pwn.log.progress("locally breaking legendre RNG seed") as plog:
            if (sol_seed := solver_func(
                rng_stream,
                p,
                # max_sample_size=max_sample_size,
                # max_samples=max_samples,
                # use_int_dict=use_int_dict,
            )) is None:
                return p, rng_stream, None
            sol_offset = sol_seed + rng_stream_len
        
        with pwn.log.progress("Attempting cadidate as solution") as plog:
            sol_key = "".join(str(b) for b in gen_rng_stream(p, sol_offset, KEY_SUBMISSION_SIZE))
            plog.status(f"{sol_key=}")
            if (flag := submit_guess(con, sol_key)) is None:
                plog.failure(f"Flag was the false flag 😱! {FALSE_FLAG=}")
                return p, rng_stream, sol_key

            plog.success(f"{flag=}")
            return flag

In [ ]:
pwn.log.setLevel("debug")

if WITH_HEAVY_ASSERTS:
    res = solve_chal(
        queries=200, # 1_000, # 10_000, # 800,
        # use_int_dict=True,

        run_local_debug=True,
        # ip="154.57.164.66", 
        # port=31866,
    )

In [ ]:
def test_run():
    rng_bits, p_bits = (
        10, 20
        # 15, 30
        # 20, 40
        # 25, 50 
        # 10, 40
    )

    p = random_prime(1 << p_bits)
    res = solve_rng(
        (rng_stream := tuple(tqdm(gen_rng_stream(p, (k_secret := randint(1, p-1)), (rng_size := 1 << rng_bits)), total=int(rng_size)))),
        p,
        max_samples = 1 << 10
    )
    
    display(
        legendre_symbol(p-1, p),
        k_secret
    )

    Zpp = GF(p)
    size = 1 << 10
    a = -23
    shift = 19
    flip = (1 - legendre_symbol(Zpp(a), p)) // 2
    block_size = ceil((size - shift) / abs(a))
    
    display(table([
        obs := tuple(flip ^^ i for i in rng_stream[:size][shift::abs(a)][::(-1 if a < 0 else 1)]),
        exp := tuple(gen_rng_stream(p, Zpp(k_secret + shift) / Zpp(a), block_size, backwards=(a < 0)))[::(-1 if a < 0 else 1)],
        exp_2 := tuple(gen_rng_stream(p, idx := ((Zpp(k_secret + shift) / Zpp(a)) - (block_size - 1 if a < 0 else 0)), block_size)),
    ]))
    return obs == exp == exp_2

if WITH_HEAVY_ASSERTS:
    assert test_run()

# Extension

In [ ]:
# WITH_HEAVY_ASSERTS = False

## Better Subsequences

In [ ]:
from sage.data_structures.bounded_integer_sequences import BoundedIntegerSequence
from typing import Generator, Sequence, Iterable

def parse_stream_to_bseq(it: Iterable) -> BoundedIntegerSequence:
    return BoundedIntegerSequence(2, list(it))

def sliding_window_seq[T: Sequence](seq: T, n: int) -> Generator[T, None, None]:
    for i in range(len(seq) + 1 - n):
        yield seq[i:i+n]

display(
    list(sliding_window_seq(_bseq := parse_stream_to_bseq([1,0,1,1,0,1]), 5)),
    _bseq.bound(),
)
del _bseq

In [ ]:
import numpy as np

from typing import Generator, Sequence, Iterable, TypeAlias
from numpy.typing import NDArray


ArrSeq: TypeAlias = NDArray[np.uint8]

DEFAULT_BIT_ORDER = 'big'

def parse_stream_to_arr(it: Iterable, size: int | None = None) -> ArrSeq:
    if size is None:
        if not isinstance(it, Sequence):
            it = tuple(it)
        size = len(it)

    it = iter(it)
    data = np.fromiter(it, dtype=np.uint8, count=size)
    try:
        next(it)
    except StopIteration:
        pass
    return data

def split_column_arr[T](arr: NDArray[T], step: int) -> Generator[NDArray[T], None, None]:
    size = len(arr)
    overflow = ((-size) % step)
    matrix = np.pad(arr, (0, overflow), mode='empty').reshape(step, -1, order='F').copy(order="C") #  ,mode='constant', constant_values=255
    for idx, col in enumerate(matrix):
        if idx >= (step - overflow):
            col = col[:-1]
        yield idx, col

def sliding_window_arr[T](arr: NDArray[T], win_size: int) -> Generator[NDArray[T], None, None]:
    if len(arr) < win_size: return
    yield from np.lib.stride_tricks.sliding_window_view(arr, win_size)

def neg_arr(arr: ArrSeq) -> ArrSeq:
    return np.bitwise_xor(arr, 1)

def arr_to_int(arr: ArrSeq) -> int:
    return int.from_bytes(np.packbits(arr, bitorder=DEFAULT_BIT_ORDER).tobytes(), DEFAULT_BIT_ORDER)

def int_to_arr(i: int, size: int) -> ArrSeq:
    return np.unpackbits(np.frombuffer(i.to_bytes(size, DEFAULT_BIT_ORDER), dtype=np.uint8), bitorder=DEFAULT_BIT_ORDER)

def gen_rng_arr(p, offset, size, backwards=False):
    return parse_stream_to_arr(gen_rng_stream(p, offset, size, backwards=False), size)


display(
    size := 133,
    stride := 7,
    table([[
        k,
        v,
        v_bits := np.unpackbits(v),
        h := arr_to_int(v_bits),
        int_to_arr(h, ceil(size / stride))
    ] for (k, v) in split_column_arr(np.arange(size, dtype=np.uint8), stride)]),
    (4 << (8*2)) + (8 << (8*1)) + (12 << (8*0)),
    data := gen_rng_arr(random_prime(1 << 50), 31337, 1 << 6),
    data_int := arr_to_int(data),
    hash(data_int),
    neg_arr(data)
)

## Better Subsequences Views

In [ ]:
from enum import IntEnum
from dataclasses import dataclass
from typing import Generator
from functools import cached_property

from sage.rings.finite_rings.finite_field_prime_modn import FiniteField_prime_modn

k = var('k', latex_name=r'k^\prime')

@dataclass(frozen=True, slots=True)
class StreamViewCase:
    is_neg: bool
    scale: int
    shift: int
    offset: int

    @property
    def sign(self):
        return (-1 if self.is_neg else 1)

    def gen_from_subseq(self, seq: Sequence, field: FiniteField_prime_modn) -> Generator[int, None, None]:
        flip = (1 - legendre_symbol(field(self.scale * self.sign), field.order())) // 2
        yield from (flip ^^ i for i in seq[self.shift::self.scale][::self.sign][self.offset:])

    def _latex_(self):
        return (
            r"\left[\begin{array}{c|c}"
            r"\operatorname{Sign}(\cdot) & " + (sign := (r"\ominus" if self.is_neg else r"\oplus")) + r" \\"
            r"|\textit{d}| & " + latex(self.scale) + r" \\"
            r"\textit{i} & " + latex(self.shift) + r" \\"
            r"\varepsilon_\textit{L} & " + latex(self.offset) + r" \\" 
            r"\end{array}\right]"
        )

@dataclass(frozen=True)
class StreamView:

    case: StreamViewCase
    M: int
    Fp: FiniteField_prime_modn

    @cached_property
    def L(self):
        return ceil((self.M - self.case.shift) / self.case.scale)

    @cached_property
    def view_suffix_size(self):
        return int(self.L - self.case.offset)

    def __len__(self):
        return self.view_suffix_size

    @cached_property
    def k(self):
        return (
            ((k + self.case.shift) / (self.case.sign * self.case.scale))
            + self.case.offset
            - (self.L - 1 if self.case.is_neg else 0)
        )

    @cached_property
    def k_fp(self):
        return self.k.polynomial(base_ring=self.Fp)

    def gen_from_subseq(self, seq: Sequence) -> Generator[int, None, None]:
        return self.case.gen_from_subseq(seq, self.Fp)

    def gen_from_k(self, k_seed: int, size: int):
        return gen_rng_stream(self.Fp.order(), self.k_fp.subs(k_seed), size)

    def _latex_(self):
        return (
            r"\left[\begin{array}{c|c}"
            r"\text{CTX} & " + latex(self.case) + r" \\"
            r"\hline"
            r"\|\textit{M}\| & " + latex(self.M) + r" \\"
            r"\|\textit{L} + \varepsilon\| & " + latex(self.L) + r" \\"
            r"\hline"
            r"\{ \cdot \}_\textit{L} & \left\{" + (
                (r"- \left( " if self.case.is_neg else "")
                + r"\frac{" + latex(k + self.case.shift) + r"}{" + latex(self.case.scale) + r"}"
                + (" + " + latex(
                        self.view_suffix_size - 1
                        if self.case.is_neg
                        else self.case.offset
                    ) if self.case.offset else ""
                ) + (r" \right)" if self.case.is_neg else "")
            ) + r"\right\}_{" + latex(self.view_suffix_size)  + r"} \\"
            r"\textit{k}^\prime \in \mathbb{Q} & " + latex(self.k) + r" \\"
            r"\hline"
            r"\mathbf{F}_p & " + latex(self.Fp) + r" \\"
            r"\textit{k} \in \mathbf{F}_p & " + latex(self.k_fp) + r" \\"
            r"\end{array}\right]"
        )

display(
    _case := StreamViewCase(
        scale=9,
        is_neg=True,
        shift=7,
        offset=68,
    ),
    _view := StreamView(
        case=_case,
        M=1024,
        Fp=GF(582041651),
    ),
    len(_view)
)
del _case, _view

## Better generators

In [ ]:
from itertools import chain, islice
from tqdm.auto import tqdm
from functools import cached_property

from dataclasses import dataclass
from typing import TypeAlias

Sample: TypeAlias = tuple[BoundedIntegerSequence, StreamViewCase]

@dataclass(frozen=True, slots=True)
class LegendreScalars:
    scale: int
    is_flip: bool
    jump: int

    def __post_init__(self):
        object.__setattr__(self, "scale", int(self.scale))
        object.__setattr__(self, "jump", int(self.jump))

@dataclass(frozen=True)
class LegendreSampler:
    p: int
    L: int
    gen_len: int = int(1 << 20) # int(500_000)

    def __post_init__(self):
        object.__setattr__(self, "p", int(self.p))
        object.__setattr__(self, "L", int(self.L))
        object.__setattr__(self, "gen_len", int(self.gen_len))

    @cached_property
    def Fp(self):
        return GF(self.p)

    @cached_property
    def scalar_symbols(self):
        return tuple(
            LegendreScalars(
                scale=scale,
                is_flip=(legendre_symbol(scale, self.p) == -1),
                jump=scale * self.L
            ) # self.Fp(scale)
            for scale in chain([1], primes(2, ceil(self.gen_len / self.L)))
        )

    @cached_property
    def idx_size(self):
        return int(sum(scalar.scale for scalar in self.scalar_symbols))

    @cached_property
    def bit_size(self):
        return self.idx_size * self.L

    @cached_property
    def gain(self):
        return self.bit_size / self.gen_len

    def sample_from_bseq(
        self,
        bseq: BoundedIntegerSequence,
        *,
        show_tqdm: int = int(0),
    ) -> Generator[Sample, None, None]:
        assert len(bseq) == self.gen_len, f"Mis-sized bseq passed! {self.gen_len=}, {len(bseq)=}, {bseq=}"
        
        ## Even though all the different scales of `q` have their own unique
        ## `legendre_symbol(q, p)` values, since q != 0, it must resolve to
        ## =>  1, i.e. the sequence values remains unchanged
        ## => -1, i.e. the sequence values must further be negated
        ## ergo, `neg_seq` plays the same role as `seq`
        ## just for cases where `legendre_symbol(q, p) == -1`
        neg_bseq = parse_stream_to_bseq(1 - bit for bit in bseq)

        yield from tqdm((
            (
                cur_seq[shift:scalar.jump:scalar.scale],
                StreamViewCase(
                    scale=scalar.scale,
                    is_neg=False,
                    shift=shift,
                    offset=0,
            ))
            for scalar in tqdm(
                self.scalar_symbols,
                desc="Subsequence scalars",
                position=int(1),
                leave=False,
                disable=(show_tqdm < 1)
            )
            if (cur_seq := (neg_bseq if scalar.is_flip else bseq)) is not None
            for shift in tqdm(
                range(scalar.scale),
                desc="Subsequence sub-scalar shift",
                position=int(2),
                leave=False,
                disable=(show_tqdm < 2)
            )
        ),
            desc=f"Yielding L-size sample subsequences ({self.L=})",
            total=self.idx_size,
            position=int(0),
            leave=False,
            disable=(show_tqdm < 0),
        )
        

    def sample_from_idx(
        self,
        idx: int,
        *,
        show_tqdm: int = int(0),
    ) -> Generator[Sample, None, None]:
        return self.sample_from_bseq(
            parse_stream_to_bseq(tqdm(
                gen_rng_stream(self.p, idx, self.gen_len),
                desc=f"Generating source sequence for {idx=}",
                total=int(self.gen_len),
                position=int(0),
                leave=False,
                disable=(show_tqdm < 0)
            )),
            show_tqdm=show_tqdm,
        )

    def gen_samples(
        self,
        max_idx_samples: int | None = None,
        *,
        show_tqdm: int = int(0),
    ) -> Generator[tuple[int, Sample], None, None]:
        if max_idx_samples is not None:
            max_idx_samples = int(max_idx_samples)

        yield from tqdm((
                (idx, sample)
                for idx in tqdm(
                    islice(gen_rand_offsets(self.p), max_idx_samples),
                    desc="Source sequence at randomized idx",
                    position=int(1),
                    total=max_idx_samples,
                    leave=False,
                    disable=(show_tqdm < -1 or max_idx_samples is None),
                )
                for sample in self.sample_from_idx(idx, show_tqdm=show_tqdm)
            ),
            desc="Yielding samples",
            position=int(0),
            total=(
                self.idx_size * max_idx_samples
                if max_idx_samples is not None
                else None
            ),
            leave=True,
            disable=(show_tqdm < -2),   
        )

    def wrap_case(self, case: StreamViewCase) -> StreamView:
        return StreamView(
            case=case,
            M=self.gen_len,
            Fp=self.Fp
        )

    def _latex_(self):
        return (
            r"\left \{ \frac{ \cdot } { "
            + latex(self.p)
            + r"} \right \}_{"
            + latex(self.gen_len)
            + r"} \mapsto "
            + r"\left \{ "
            + r"\left \{ \frac{ \cdot } { "
            + latex(self.p)
            + r"} \right \}_{"
            + latex(self.L)
            + r"}"
            + r"\right \}_{"
            + latex(self.idx_size)
            + r"} (\approx "
            + self.gain
            + r")"
        )

display(
    LegendreSampler(p=582041651, L=32),
)

In [ ]:
# sampler = LegendreSampler(p=p, L=97)
# all(sampler.sample_from_idx(1000))
# all(sampler.gen_samples(1))
# all(hash(s) for _, s, _ in LegendreSampler(p, 50, gen_len=100_000).gen_samples(10)) # 500_000 # 1_000_000

def test_sampler():
    sampler = LegendreSampler(p=random_prime(1 << 39, 1 << 40), L=97, gen_len=100_000)
    cases = {}
    for idx, (subseq_exp, case) in sampler.gen_samples(1):
        view = sampler.wrap_case(case)
        if subseq_exp != (subseq_obs := parse_stream_to_bseq(view.gen_from_k(idx, sampler.L))):
            display(
                table([subseq_exp, subseq_obs]),
                idx,
                view,
            )
            raise RuntimeError("Self-contradicting result found!")
    
        if (case := (idx, subseq_exp)) in cases:
            print("Overlap found!")
            if idx != (view.k_fp - cases[case].k_fp).any_root():
                raise RuntimeError("Repeated view found!")
    
        cases[case] = view
    return True
    

if WITH_HEAVY_ASSERTS:
    assert test_sampler()

## Better Dictionaries

In [ ]:
# <https://docs.python.org/3/library/collections.abc.html#collections-abstract-base-classes>
from collections.abc import Mapping
from typing import Self

from dataclasses import dataclass

from tqdm.auto import tqdm
from itertools import islice

from rbloom import Bloom
from pyfusefilter import Fuse16

DEFAULT_MAX_L = 256

@dataclass(frozen=True)
class LegendreRngDict(Mapping):
    p: int
    L: int
    source: BoundedIntegerSequence
    table: dict[BoundedIntegerSequence, StreamViewCase]
    bloom_prob: int = 0.01

    @cached_property
    def M(self):
        return len(self.source)

    @cached_property
    def Fp(self):
        return GF(self.p)

    def wrap_case(self, case: StreamViewCase) -> StreamView:
        return StreamView(
            case=case,
            M=self.M,
            Fp=self.Fp
        )

    # TODO: replace this with a Fuse16 variant that doesn't have a bad hash function
    # Also, experiement with rBloom first to see if this is even useful or not
    @cached_property
    def bloom_filter(self) -> Bloom:
        print("Building Bloom Filter")
        prob_filter = Bloom(len(self), self.bloom_prob)
        prob_filter.update(self.table.keys())
        print(f"Bloom Filter built! {prob_filter=}")
        return prob_filter

    # TODO: replace this with a Fuse16 variant that doesn't have a bad hash function
    # Also, experiement with rBloom first to see if this is even useful or not
    @cached_property
    def fuse_filter(self) -> Fuse16:
        print("Building Fuse16 Filter")
        prob_filter = Fuse16(len(self))
        prob_filter.populate(self.table.keys())
        # prob_filter = Bloom(len(self), 0.01)
        # prob_filter.update(self.table.keys())
        print(f"Fuse16 Filter built! {prob_filter=}")
        return prob_filter

    @classmethod
    def expect_scalars(cls, m, l, as_int=True):
        exp = (m - 1) / (l - 1)
        if as_int:
            return int(floor(exp))
        return exp

    @classmethod
    def expect_size(cls, m, l, parts=False, as_int=True):
        """given a continuous rng_stream sequence and the subsequence length, return the expected number of derivable substrings

        Source: <https://eprint.iacr.org/2020/098.pdf#page=7>
        """
        l_d = cls.expect_scalars(m, l, as_int=as_int)
        total = 2*m*l_d
        exceptions = (l-1)*l_d*(l_d+1)

        if as_int:
            exceptions = int(exceptions)
            total = int(total)
        if parts:
            return exceptions, total
        return total - exceptions        

    @classmethod
    def min_L(cls, m, max_L=DEFAULT_MAX_L, strict=True) -> int:
        """Calculates the minimum L size based on the pidgenhole principle

        If strict, gives the direct bound (i.e. against N := 2^L spaces)
        If loose, gives the bound factoring in the birthday paradox
            (i.e. against (N := 2^L)^2 = 2^2L spaces - since to avoid collisions, you need ~N^2 space)
        """
        M, L = var('M, L')
        lhs = cls.expect_size(M, L, as_int=False)
        expr = (lhs - (2**(L if strict else L/2))).expand() # 
        return int(ceil(find_root(expr.subs(M=m) == 0, 1, max_L)))

    @classmethod
    def gen_seq_subviews(cls, bseq: BoundedIntegerSequence, p: int, L: int, no_tqdm=False) -> Generator[Sample, None, None]:
        Fp = GF(p)
        size = len(bseq)

        neg_bseq = parse_stream_to_bseq(1 - bit for bit in bseq)
        is_neg_flip = (legendre_symbol(Fp(-1), p) == -1)

        for scale in tqdm(
            range(1, 1 + cls.expect_scalars(size, L, as_int=True)),
            position=0,
            desc="Substring scalars",
            disable=no_tqdm
        ):
            is_scale_flip = (legendre_symbol(scale, p) == -1)
            
            for is_neg, sign in tqdm(
                ((False, int(1)), (True, int(-1))),
                position=1,
                leave=False,
                desc="Substring reflection",
                disable=no_tqdm
            ):
                cur_seq = (neg_bseq if (is_scale_flip != (is_neg_flip and is_neg)) else bseq)
                yield from ((
                    subseq,
                    StreamViewCase(
                        scale=scale,
                        is_neg=is_neg,
                        shift=shift,
                        offset=i,
                        # M=size,
                        # Fp=Fp,
                    ))
                    for shift in tqdm(range(scale), position=2, leave=False, desc="Substring sub-scalar shift", disable=no_tqdm)
                    for i, subseq in enumerate(sliding_window_seq(cur_seq[shift::scale][::sign], L))
                )

    # TODO: replace the messy verbosity system with a proper logging system
    @classmethod
    def build_subview_dict(
        cls,
        it,
        p,
        *,
        max_L=DEFAULT_MAX_L,
        strict_L=False,
        # Visuals
        clear_bar=True,
        verbosity=0,
    ) -> Self | int | None:
        Fp = GF(p)
        seq = (parse_stream_to_bseq(it) if not isinstance(it, BoundedIntegerSequence) else it)
        m = len(seq)
        min_len = cls.min_L(m, strict=strict_L)
        
        if verbosity >= 0: display(f"{Fp}, {m=}, {min_len=}")
    
        l_bound = 0
        prev_overlap_seeds = set()
        for l in (outer_tqdm := tqdm(
            range(min_len, max_L + 1),
            desc="Trialling substring lengths for unique index dictionary",
            initial=int(min_len),
            total=int(max_L),
            maxinterval=5,
            disable=(verbosity < -2),
        )):
            if l_bound >= l:
                if verbosity >= 1: display(f"Skipped iteration {l=}")
                continue
    
            subseq_total = cls.expect_size(m, l)
            subseq_dict = {}
            for subseq, case in (inner_tqdm := tqdm(
                cls.gen_seq_subviews(
                    bseq=seq,
                    p=p,
                    L=l,
                    no_tqdm=True
                ),
                desc=f"Trial - |L|={l:#02x}",
                total=subseq_total,
                disable=(verbosity < -1),
            )):
                if ((prev_case := subseq_dict.setdefault(subseq, case)) is case):
                    # subseq_dict[subseq] = case
                    continue

                # Constradiction was found, and inner loop will be exited - I am doing deletions
                # pre-emptively and explicitly just to absolutely make sure I have as little wasted memory
                del subseq_dict

                prev_view = StreamView(
                    case=prev_case,
                    M=m,
                    Fp=Fp,
                )
                cur_view = StreamView(
                    case=case,
                    M=m,
                    Fp=Fp,
                )
    
                if clear_bar and not inner_tqdm.disable: tqdm_remover(inner_tqdm)
                if verbosity >= 2: display(table([[subseq, prev_view, cur_view]]))
                
                if 1 != (
                    overlap := (prev_view.k_fp - cur_view.k_fp)
                ).degree():
                    if verbosity >= 1: display(f"Local overlap found - but {l=} is broken, with {overlap=}...")
                    break
                    
                overlap_seed = overlap.any_root()
                if verbosity >= 1: display(f"Local overlap found - {l=}, {overlap_seed=}")

                if (
                    overlap_seed not in prev_overlap_seeds
                    and all(
                        exp == obs
                        for exp, obs
                        in zip(seq, gen_rng_stream(p, overlap_seed, len(seq)))
                )):
                    if verbosity >= 0: display(f"SEED FOUND! {overlap_seed=}")
                    return int(overlap_seed)
    
                prev_overlap_seeds |= {overlap_seed}
                if verbosity >= 0: display(f"False seed found - {l:#02x} / {overlap_seed=}")

                if (
                    (mismatch_idx := next((
                        idx
                        for idx, (b0, b1)
                        in enumerate(zip(prev_view.gen_from_subseq(seq), cur_view.gen_from_subseq(seq)))
                        if b0 != b1
                    ), None)) is not None
                    and (skipper := mismatch_idx - l) > 0
                ):
                    if verbosity >= 0: display(f"Overlap size yields L-jump {l:#02x} ↦ {mismatch_idx+1:#02x} - skipping {skipper} intermediate iterations!")
                    l_bound = mismatch_idx

                break
            else:
                if verbosity >= 0: display(f"View dictionary successfully built!")
                return cls(
                    p=p,
                    L=l,
                    source=seq,
                    table=subseq_dict,
                )
        return None
    
    def assert_from_source(self):
        assert len(self) == (
            len_exp := self.expect_size(self.M, self.L)
        ), f"LegendreRngDict Source Size Failure - {len_exp=}, {len(self)=}"
        
        for k_exp, v in tqdm(self.items(), desc="Running source validation asserts"):
            assert k_exp == (
                k_obs := parse_stream_to_bseq(islice(self.wrap_case(v).gen_from_subseq(self.source), self.L))
            ), f"LegendreRngDict Source Validation Failure - {k_exp=}, {k_obs=}, {v=}"
    
    def assert_from_key(self, k_val):
        for k_exp, v in tqdm(self.items(), desc="Running key validation asserts"):
            assert k_exp == (
                k_obs := parse_stream_to_bseq(self.wrap_case(v).gen_from_k(k_val, self.L))
            ), f"LegendreRngDict Key Validation Failure - {k_exp=}, {k_obs=}, {v=}"
    
    def assert_fuse_filter(self):
        for k_exp in tqdm(self.keys(), desc="Running filter validation asserts"):
            assert k_exp in self.fuse_filter, f"LegendreRngDict Filter Validation Failure - {k_exp=}, {self.fuse_filter=}"

    def gen_overlaps(self, max_samples: int | None):
        max_samples = int(max_samples)

        yield from (
            (cand_idx, self.wrap_case(case_match))
            for cand_idx in tqdm(islice(gen_rand_offsets(p), max_samples), total=max_samples)
            if (case_match := self.get(parse_stream_to_bseq(gen_rng_stream(p, cand_idx, self.L)))) is not None
        )

    def gen_overlaps_from_sampler(
        self,
        sampler: LegendreSampler,
        max_idx_samples: int | None = None,
        *,
        show_tqdm: int = int(0),
        skip_fuse = True,
        skip_bloom = True,
    ):
        assert all((
            sampler.Fp == self.Fp,
            sampler.p == self.p,
            sampler.L == self.L,
        ))

        yield from tqdm((
                (sampler.wrap_case(case).k_fp(idx), self.wrap_case(case_match))
                for idx in tqdm(
                    islice(gen_rand_offsets(self.p), max_idx_samples),
                    desc="Sampling `idx`",
                    position=int(1),
                    total=max_idx_samples,
                    leave=False,
                    disable=(show_tqdm < -1 or max_idx_samples is None),
                )
                for subseq, case in sampler.sample_from_idx(idx, show_tqdm=show_tqdm)
                if skip_fuse or (subseq in self.fuse_filter)
                if skip_bloom or (subseq in self.bloom_filter)
                if (case_match := self.get(subseq)) is not None
            ),
            desc="Yielding samples",
            position=int(0),
            # total=(
            #     sampler.idx_size * max_idx_samples
            #     if max_idx_samples is not None
            #     else None
            # ),
            leave=True,
            disable=(show_tqdm < -2),   
        )

    def gen_overlaps_fast(
        self,
        gen_len: int | None = None,
        **kwargs
    ):
        return self.gen_overlaps_from_sampler(
            LegendreSampler(
                p=self.p,
                L=self.L,
                **{ k: v
                    for k, v
                    in (
                        ("gen_len", gen_len),
                    ) if v is not None
                }
            ),
            **kwargs
        )
    
    # <https://docs.python.org/3/library/collections.abc.html#collections.abc.Mapping>
    def __getitem__(self, x):
        return self.table[x]

    def __iter__(self):
        return iter(self.table)

    def __len__(self):
        return len(self.table)

    def _latex_(self):
        return (
            r"\left \{ \frac{ \cdot } { "
            + latex(self.p)
            + r"} \right \}_{"
            + latex(self.L)
            + r"} \xtwoheadrightarrow[\bot]{\text{lookup}} " # \textcolor{red}{}
                # \xhookrightarrow{\text{lookup}}
                # \overset{\text{lookup}}{\rightharpoonup}
                # \xmapsto{\text{lookup}}
                # \leadsto \rightsquigarrow \hookrightarrow
            + r"\left[" # \{\bot\} \cup # \text{Option} \left( 
            + r"\frac{\textit{k} + \textit{i}}{\textit{d}} \right]_{"
            + r"\varepsilon_\textit{L} }^{"
            + latex(self.L)
            + r" + \varepsilon_\textit{L} }" # \right)
        )

display(
    LegendreRngDict.build_subview_dict(
        gen_rng_stream(
            _p := 1112707506698441, # random_prime(1 << 50)
            _k_secret := 1234567890, # randint(1, p-1)
            _rng_size := 1 << 10,
        ),
        _p,
        verbosity=-3,
        strict_L=False
    )
)

In [ ]:
def test_dictionary(p, k_seed):
    rng_len = len(rng_substream := tuple(gen_rng_stream(p, k_seed, (1 << 10))))
    
    match LegendreRngDict.build_subview_dict(rng_substream, p, strict_L=True):
        case None:
            raise RuntimeError("Failed to build dictionary!")
    
        case int(seed):
            assert k_seed == seed, f"BAD SEED - exp={k_seed} obs={seed}"
            display(f"Seed recovered - {seed=} {rng_len=}")
            assert False, "Pre-emptive solution"
    
        case LegendreRngDict() as windows:
            win_len = len(windows)
    
            display(f"{rng_len=}, {win_len=}, {windows.L=}")
            if WITH_HEAVY_ASSERTS:
                windows.assert_from_source()
                windows.assert_from_key(k_seed)
                windows.assert_fuse_filter()
    
            size = 50
            display(table(win_slice := list(islice(enumerate(windows.values()), off := (win_len - size) // 2, off + size))).transpose())

    return True

set_random_seed(31337)
assert test_dictionary(
    _p := random_prime(1 << 40),
    _k_seed := randint(1, _p-1)
)

## Better Solvers



Good pattern matching guide for python - <https://peps.python.org/pep-0636/>

In [ ]:
import pwn
from itertools import islice

def solve_rng_beefy(
    rng_stream,
    p,
    /,
    max_L: int = int(1 << 10),
    max_idx_samples: int = int(1 << 10),
    gen_len: int | None = None,
) -> int | None:
    rng_stream_len = len(rng_stream)

    with pwn.log.progress("Building rng_stream window dictionary") as plog:
        match LegendreRngDict.build_subview_dict(rng_stream, p, max_L=max_L, verbosity=0):
            case None:
                plog.failure(f"key stream dict could not be built - loosen the {max_L=}")
                return None
            case int(seed):
                plog.success(f"Key stream dictionary construction derived a seed-recovering overlap!")
                return seed
            case LegendreRngDict() as rng_views:
                plog.success(f"Successfully built rng_stream window dictionary!")
                display(latex(rng_views))
            case _ as res:
                raise RuntimeError(
                    f"This should never happen - types should have exhaustively narrowed value into LegendreRngDict {res=}, {type(res)=}"
                )

    with pwn.log.progress("Offset candidate trials") as plog:

        prev_cand = set()
        for idx, view in rng_views.gen_overlaps_fast(gen_len=gen_len, max_idx_samples=max_idx_samples, skip_bloom=False):

            if not (cand := (view.k_fp - idx)).degree():
                plog.status(f"Candidate view had zero-degree ({cand=})- dismissing...")
                continue

            cand_seed = cand.any_root()
            if cand_seed in prev_cand:
                plog.status(f"Repeated candidate found :-( - {cand_seed=}")
                continue

            plog.status(f"New candidate found! {cand_seed=}")
            if any(
                exp != obs
                for exp, obs
                in zip(rng_stream, gen_rng_stream(p, cand_seed, rng_stream_len))
            ):
                prev_cand |= {cand_seed}
                continue
            
            plog.success(f"Candidate is a match! {cand_seed=}")
            display(view)
            return cand_seed
        else:
            plog.failure("We could not find a solution! increase max_idx_samples/gen_len, or collect more bits")
            return None

In [ ]:
def solve_chal_beefy(*args, **kwargs):
    return solve_chal(*args, **kwargs, solver_func=solve_rng_beefy)

if WITH_HEAVY_ASSERTS:
    solve_chal_beefy(
        queries=25, # 10_000, # 200, # 800,
    
        run_local_debug=True,
        # ip="154.57.164.66",
        # port=31866,
    )

In [ ]:
rng_size, p_bits = (
    # 1 << 10, 20
    # 1 << 15, 30
    # 1 << 20, 40
    # 1 << 25, 50 
    #
    # 1 << 11, 34
    # 
    # 1 << 10, 30 
    # 1 << 12, 40
    # 1 << 14, 40
    # 1 << 14, 48
    # 1 << 14, 50
    # 3 << 13, 48
    # 3 << 13, 50
    3 << 14, 56
    #
    # 1 << 15, 50
    # 1 << 16, 50

    # Above 1 << 14 bits of RNG material, the extracted substring dictionary actually hits python's
    # memory threshold on my laptop, and construction speed massively drops from ~100_000 it/s to ~3 it/s :-(
    # IDEA: use a probabilisitc filter (bloom, cuckoo, XOR, etc) to identify new elements quickly and w/o touching
    # frequent lookups in the acutal massive subview dictionary.
    # 
    # Split probabilstic filter and actual dictionary into two separate objects; the former for quick pre-emptive filtering,
    # the latter for actual deep checks and view look-up when matches are found
)

display(
    p := random_prime(1 << p_bits),
    log(p, 2).n(),
    k_secret := randint(1, p-1),
    rng_size # := 1 << rng_bits
)

assert (
    k_secret == (k_obs := solve_rng_beefy(
        (rng_stream := tuple(tqdm(
            gen_rng_stream(p, k_secret, rng_size),
            total=int(rng_size),
            desc="Creating simulated RNG stream")
        )),
        p
    ))
), f"{k_secret=}, {k_obs=}"

print("HOLDS!!!")

# Ideas

In [ ]:
[_ for _ in tqdm(LegendreSampler(p=p, L=97, gen_len=1_000_000).gen_samples(10))]

## Done

- Try perhaps using a proper bit-level data structure?
    - <https://doc.sagemath.org/html/en/reference/data_structures/sage/data_structures/bitset.html>
    - <https://doc.sagemath.org/html/en/reference/data_structures/sage/data_structures/bounded_integer_sequences.html>
- Other very cool sagemath data structures:
    - <https://doc.sagemath.org/html/en/reference/data_structures/sage/data_structures/stream.html>
    - <https://doc.sagemath.org/html/en/reference/data_structures/sage/misc/binary_tree.html>

## TODO

Legendre papers:
- <https://eprint.iacr.org/2021/182.pdf>

---

Probabalistic filters:
- <https://en.wikipedia.org/wiki/Bloom_filter>
    - <https://www.geeksforgeeks.org/python/bloom-filters-introduction-and-python-implementation/>
    - <https://systemdesign.one/bloom-filters-explained/>
    - <https://hur.st/bloomfilter/>
    - <https://github.com/KenanHanke/rbloom>
        - <https://github.com/prashnts/pybloomfiltermmap3>
        - <https://github.com/barrust/pyprobables>

---

- <https://en.wikipedia.org/wiki/Cuckoo_filter>
    - <https://en.wikipedia.org/wiki/Cuckoo_hashing>

- <https://lemire.me/blog/2019/12/19/xor-filters-faster-and-smaller-than-bloom-filters/>


- <https://github.com/FastFilter>
    - <https://github.com/FastFilter/pyfusefilter>
    - <https://arxiv.org/abs/2201.01174>

---

- <https://web.stanford.edu/class/archive/cs/cs166/cs166.1216/>
    - <https://web.stanford.edu/class/archive/cs/cs166/cs166.1216/lectures/13/Slides13.pdf>

 ---

Cool things to look into

 - <https://en.wikipedia.org/wiki/MurmurHash#Vulnerabilities>
     - <https://en.wikipedia.org/wiki/SipHash>
     - <https://en.wikipedia.org/wiki/List_of_hash_functions#Non-cryptographic_hash_functions>

 
 - <https://doc.sagemath.org/html/en/reference/combinat/sage/combinat/bijectionist.html>

     - <https://doc.sagemath.org/html/en/reference/combinat/sage/rings/cfinite_sequence.html>
  
---

- <https://stackoverflow.com/questions/55824130/is-it-possible-to-create-a-minimal-perfect-hash-function-without-a-separate-look>
    - <https://en.wikipedia.org/wiki/Perfect_hash_function>
    - <https://stevehanov.ca/blog/?id=119>
        - <https://cmph.sourceforge.net/>
        - <https://engineering.indeedblog.com/blog/2018/02/indeed-mph/>
        - <https://github.com/jermp/pthash>
        - <https://github.com/rizkg/BBHash>
            - <https://github.com/dib-lab/pybbhash>
        - <https://github.com/ilanschnell/perfect-hash>
    - <https://arxiv.org/html/2406.18099v1>
    - n/a, but still cool - <https://github.com/codenotary/immudb>

### Tasks:
- <https://github.com/FastFilter> seems legit
    - Experiment with <https://github.com/FastFilter/pyfusefilter>
    - Consider using this to augment the current hash map approach

sketched idea:
- <https://github.com/KenanHanke/rbloom> for filtering during construction, since it is updatable and mutable
    - Need to experiment with overlap-check-then-build, vs live building in batches, vs current live building and overlap checking
 

- <https://github.com/FastFilter/pyfusefilter> for filtering once done, since it is immutable, size-initialisable, very memory efficient and very fast
    - Can use this as the life candidate filter accelerator for making candidiate filtering as fast as possible
    - <https://github.com/FastFilter/pyfusefilter/blob/c676938cb0a90e374a575f01d064cddda4024e0a/pyfusefilter/pyfusefilter.py#L7> is a little sus - what about forking it so it can accept bytes as input?
        - Also, the closest I can get `bseq` to directly accessing its underlying data is `bytes()`, which still pads it out; perhaps I can build a child object that can expose its underlying integer representation as well?
        - Correction - I think its hash is literally just `capacity * 1073807360 + data_in_lsb_order % ((((1 << 32) - 1) << 31) \approx (1 << 62))`
            - <https://github.com/sagemath/sage/blob/4015f9189b6aa95f46bb957e78d303f5fb20e77e/src/sage/data_structures/bounded_integer_sequences.pyx#L202>
            - <https://github.com/sagemath/sage/blob/4015f9189b6aa95f46bb957e78d303f5fb20e77e/src/sage/data_structures/bitset_base.pxd#L630>
        - Ergo, just find a way to do this with `hash(bseq)` as the actual input

- Perhaps store the core table not as a hashmap, but as:

    - A trie?
        - <https://doc.sagemath.org/html/en/reference/combinat/sage/combinat/words/suffix_trees.html>
    - A proper out-of-core database (e.g. <https://github.com/LMDB>)

Also, for minimal overhead - why don't we move the subsequence comparison *into* the construction loop, so we don't even build a view if its substring doesn't match?

In [ ]:
%pip install xxhash rbloom pyfusefilter

In [ ]:
from sage.data_structures.bounded_integer_sequences import BoundedIntegerSequence
import xxhash

# l_d = (M - 1) / L
# expr = 2*M*l_d - ((L-1)*l_d*(l_d+1)) - 2**L
# find_root(expr.subs(M=(size := 1 << 20)) == 0, 1, 100), log(2 * size ** (4/2)).n()

# str(_bseq)
# bytes(_bseq)
# dumps(_bseq)
MAGIC_NUMBER = 1073807360
hash(BoundedIntegerSequence(2, [0,0,1])) - MAGIC_NUMBER, hash(BoundedIntegerSequence(2, [0,1,0])) - MAGIC_NUMBER

ones, zeros = 32,31 # 31,32 # 1, 62
display(
    hash(BoundedIntegerSequence(2,  ([0] * zeros) + ([1] * ones))) - 1073807360 - (((1 << ones) - 1) << zeros),
    xxhash.xxh64_intdigest(hash(_bseq).to_bytes(8)),
)

In [ ]:
from rbloom import Bloom

Bloom(10_000_000, 0.0001)


import pyfusefilter

In [ ]:
set_random_seed(31337)
dict_lrng = LegendreRngDict.build_subview_dict(gen_rng_stream(p, k_secret, 1 << 14), p)

In [ ]:
%pip install Pympler

In [ ]:
import sys
from pympler.asizeof import *

asizeof(dict_lrng), 1_553_050_920

In [ ]:
from itertools import islice
from pympler.asizeof import asized
v_example = list(islice(dict_lrng.values(), 10))
asized(v_example, detail=3).format()

In [ ]:
del dict_lrng

In [ ]:
%timeit (fuse_filter := pyfusefilter.Fuse16(dict_lrng.keys()))

In [ ]:
# fuse_filter
# pyfusefilter.Fuse16(tqdm(dict_lrng.keys()))